In [4]:
# to bring all functions that we have develop for marxan.
%run marxan_utils.ipynb

Files that we need to create by default: 
* input.dat
* input
    * puvsp.dat (PlanningUnitVSConservationFeature)
    * spec.dat (conservationFeature)
    * pu.dat (planningUnits)  
    
    --- only if certain conditions in input.dat are met.
    * bound.dat (boundaryLength) only if blm > 0 
    
    * .dat (blockDefinition) only if we group features/species

* output


## input.dat file

```sql
--input.dat
select number_of_runs, blm, metadata from scenario where id = ''
```

In [5]:
inputDatFile?

Init signature:
inputDatFile(
    *,
    VERSION: str = '0.1',
    BLM: float = 0.0,
    PROP: float = 0.0,
    RANDSEED: int = -1,
    NUMREPS: int = 1,
    BESTSCORE: int = 0,
    NUMITNS: int = 0,
    STARTTEMP: int = 1,
    COOLFAC: int = 0,
    NUMTEMP: int = 1,
    COSTTHRESH: float = 0,
    THRESHPEN1: float = 0,
    THRESHPEN2: float = 0,
    INPUTDIR: str = 'input',
    SPECNAME: str = 'spec.dat',
    PUNAME: str = 'pu.dat',
    PUVSPRNAME: str = 'puvspr2.dat',
    BOUNDNAME: str = 'bound.dat',
    BLOCKDEFNAME: str = 'blockdef.dat',
    VERBOSITY: int = 1,
    MISSLEVEL: float = 1,
    OUTPUTDIR: str = 'output',
    SCENNAME: str = 'Default_name',
    SAVERUN: int = 3,
    SAVEBEST: int = 3,
    SAVESUMMARY: int = 3,
    SAVESCEN: int = 3,
    SAVETARGMET: int = 3,
    SAVESUMSOLN: int = 3,
    SAVELOG: int = 1,
    SAVESNAPSTEPS: int = 0,
    SAVESNAPCHANGES: int = 0,
    SAVESNAPFREQUENCY: int = 0,
    SAVESOLUTIONSMATRIX: int = 3,
    RUNMODE: int = 1,
    ITIMPTYPE: int =

## pu.dat file

```sql
-- pu.dat
select puid, lockin_status as status, xloc, yloc, cost 
from scenarios_pu_data spd
INNER JOIN scenarios_pu_cost_data spcd on spd.id = spcd.scenarios_pu_data_id
INNER JOIN projects_pu ppu on ppu.id = spd.project_pu_id
where scenario_id = '<scenario_id>';
```

In [6]:
planningUnits?

Init signature:
planningUnits(
    *,
    id: int,
    cost: float = 1,
    status: int = 0,
    xloc: float = None,
    yloc: float = None,
) -> None
Docstring:     
The Planning Unit File contains all the information related to planning units, except
for the distribution of conservation features across planning units (which is held in the
Planning Unit versus Conservation Feature File ). The default name for this file is
‘pu.dat’.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /opt/conda/lib/python3.8/site-packages/pydantic/main.cpython-38-x86_64-linux-gnu.so
Type:           ModelMetaclass
Subclasses:     


## spec.dat file

```sql
-- spec.dat
select feature_id as id, target, prop, fpf as spf, target2, targetocc, sepnum, metadata->'sepdistance' as sepdistance
from scenario_features_data
where scenario_id = ''
```

In [7]:
conservationFeature?

Init signature:
conservationFeature(
    *,
    id: int,
    type: int = None,
    target: float = None,
    prop: __main__.ConstrainedFloatValue = None,
    spf: float,
    target2: float = None,
    targetocc: float = None,
    name: str = None,
    sepnum: float = None,
    sepdistance: float = None,
) -> None
Docstring:     
The Conservation Feature File contains information about each of the conservation
features being considered, such as their name, target representation, and the penalty
if the representation target is not met. It has the default name ‘spec.dat’. Because of
this name it is sometimes referred to as the Species File, although conservation
features will oftenbe surrogates such as habitat type rather than actual species. 
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /opt/conda/lib/python3.8/site-packages/pydantic/main.cp

## puvsp.dat file

```sql
-- puvsp.dat
select puid, feature_id, ST_Area(ST_Transform(st_intersection(species.the_geom, pu.the_geom),3410))  as amount
from (select the_geom, sfd.feature_id 
from scenario_features_data sfd 
inner join features_data fd  on sfd.feature_class_id = fd.id) species, (select the_geom, ppu.puid 
	from planning_units_geom pug
	inner join projects_pu ppu on ppu.geom_id = pug.id
	inner join scenarios_pu_data spd on ppu.id = spd.project_pu_id
	where spd.scenario_id = '' order by ppu.puid asc) pu 
where st_intersects(species.the_geom, pu.the_geom)
order by puid asc
```

In [8]:
planningUnitVSConservationFeatureV?

Init signature: planningUnitVSConservationFeatureV(*, species: int, pu: int, amount: float) -> None
Docstring:     
The Planning Unit versus Conservation Feature File contains information on the
distribution of conservation features across planning units. It has the default file
name, ‘puvpsr2.dat’. There are two different formats this file can take, vertical and
horizontal. Either is acceptable and Marxan will test the header line to determine
which format is being used. This one represent the vertical format
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /opt/conda/lib/python3.8/site-packages/pydantic/main.cpython-38-x86_64-linux-gnu.so
Type:           ModelMetaclass
Subclasses:     


## bound.dat file

```sql
-- bound.dat use cross join
with pu as (select the_geom, ppu.puid 
from planning_units_geom pug
inner join projects_pu ppu on ppu.geom_id = pug.id
inner join scenarios_pu_data spd on ppu.id = spd.project_pu_id
where spd.scenario_id = '')
SELECT DISTINCT a.puid id1, b.puid id2, ST_Length(st_transform(ST_CollectionExtract(ST_Intersection(a.the_geom, b.the_geom), 2)), 3410)/1000 boundary  
from pu a, pu b  
WHERE a.puid < b.puid AND ST_Touches(a.the_geom, b.the_geom);
```

In [9]:
boundaryLength?

Init signature: boundaryLength(*, id1: int, id2: int, boundary: float) -> None
Docstring:     
The Boundary Length File contains information about the length or ‘effective length’
of shared boundaries between planning units. This file is necessary if you wish to use
the Boundary Length Modifier to improve the compactness of reserve solutions (bound.dat).

!Any missing values within the file will prevent Marxan from running, for instance 
!if ‘id1’ and ‘id2’ are set but no value for ‘boundary’ is entered.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /opt/conda/lib/python3.8/site-packages/pydantic/main.cpython-38-x86_64-linux-gnu.so
Type:           ModelMetaclass
Subclasses:     
